In [295]:
import nltk

import io 
import codecs
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import stem

import re
import pandas as pd

from nltk.corpus import stopwords
from string import punctuation
import ast
from collections import Counter

import heapq
import math
from bs4 import BeautifulSoup as soup

In [42]:
def cleaner(text):  #this functions does the preprocessing work on the string given as input and returns the cleaned string

    words = re.split(r'\W+', str(text))
    words = [word.lower() for word in words]
    without_punct = [wp for wp in words if wp not in punctuation]
    sw = stopwords.words('english')
    without_sw = [w for w in without_punct if w not in sw] 
    list_to_remove = ['b','br','span', 'one' , 'id', 'none' ]
    clean_more = [w for w in without_sw if w not in list_to_remove] 
    clean_more_2 = [w for w in clean_more if  not (re.findall(re.compile(r'freetext'),w)
                                                   or re.findall(re.compile(r'\d'),w)) ]
    
    
    ps = nltk.stem.PorterStemmer()
    stemmed_list=[ps.stem(w) for w in clean_more_2 ]
    return ' '.join(stemmed_list)

In [64]:
for books in range(20001,30001):
    text_1=''
    try:
        df = pd.read_csv("books_3_tsv/article_"+str(books)+".tsv", sep='\t', encoding='utf-8' ) #vocab read csv files
    except:
        continue
    df = df.fillna("")
    df['book_title']=df['book_title'].apply(cleaner)
    df['complete_plot']=df['complete_plot'].apply(cleaner)
    #df['book_series']=df['book_series'].apply(cleaner)

    #df.to_csv('cleaned_csv/prova_libro_'+str(books)+'.csv') #to create new cleaned csv use this line 
    
    #now create the vocab
    
    title = df.iloc[0, 0]
    plot = df.iloc[0, 8]
    text_1 = title + " " + plot
    arr_1=(text_1.split())
    c = Counter(arr_1)  # create a dictionary with the number of occurences for each word
    file = open("txt_files_2/article_"+str(books)+".txt", "w")  # save it in a txt files 
    file.write(str(c)) 
    file.close() 

 #save vocabulary as csv with indices for each vocab

In [127]:
from collections import defaultdict
d = defaultdict(list)  #create a defaultdict. keys: word , values: (document id, occurrences of the word)

In [69]:
def extract_from_counter(id): #given the index of the book it reads the dictionary in txt file and returns it 
    try:
        
        file = open("txt_files_2/article_"+str(doc)+ ".txt", "r")
        dic=file.read()
        prova=dic.replace('Counter(','').replace(')','')
        test=ast.literal_eval(prova) #evaluate the dictionary saved as a string in the txt file 
        file.close()
        return test
    except:
        return {}

In [128]:
for doc in range(20000,30001):   #fill the defaultdict d with all the documents
    result=extract_from_counter(doc)
    for key in result.keys():
        heapq.heappush(d[key],(result[key],doc)) # using heap to store values and to keep the order
       

In [95]:
df_voc= pd.read_csv('vocab.csv') # read the vocabulary id: word from the csv file
df_voc.columns=['id','word'] # change columns names 

In [132]:
dic_keys=d.keys() # create a new dictionary to subsitute words with words_id
new_dic=defaultdict(list)
for item in dic_keys:
    try:
        new_key=df_voc[df_voc.word==item].id.iloc[0] # find the word id from the vocabulary
        new_dic[new_key]=d[item] # copy the values in the new dictionary
    except:
        continue

Now we collect the values to calculate the tfidf

In [148]:
#save the lenght of each document in a dictionary
dic_lenght=defaultdict(int)

In [149]:
for doc in range(20001,30001): # fill dic_lenght with all the documents lenghts 
    result=extract_from_counter(doc)
    lenght=sum(result.values()) # sum the values of the Counter of the document 
    if(lenght==0):
        continue
    dic_lenght[doc]=lenght

In [158]:
documents_number=10000  # numbers of documents in the collection

In [166]:
# save the number of documents for each word in a different dictionary
documents_with_word= defaultdict(int)
for item in d.keys():
    documents_with_word[item]=len(d[item])

    

In [173]:
#create again the dictionary with the heap ordered by tf_idf
final_inverted_index=defaultdict(list) #this is the final inverted indev with words as keys.
for doc in range(20000,30001):
    result=extract_from_counter(doc)
    doc_len=dic_lenght[doc]
    for key in result.keys():
        tf= result[key] / doc_len
        idf= math.log(documents_number / documents_with_word[key] ,10)
        tf_idf= tf * idf
        heapq.heappush(final_inverted_index[key],(tf_idf,doc)) # the list is ordered by the tfidf using a heap 

In [178]:
#change the word with the numbers  
dic_keys=final_inverted_index.keys()
inverted_index_tfidf=defaultdict(list)  #final inverted index
for item in dic_keys:
    try:
        new_key=df_voc[df_voc.word==item].id.iloc[0]
        inverted_index_tfidf[new_key]=final_inverted_index[item]
    except:
        continue

In [227]:
file= open('inverted_index_1.txt','r') #read the inverted index of the previous search engine 
obj= file.read()

In [228]:
new_dic=ast.literal_eval(obj) #evaluate string as dictionary

In [261]:
def find_tfidf(word_list): # given a list, this functions returns the tfidf_vector for the string
    c=Counter(word_list)
    res_vec=[] 
    string_len=len(word_list)
    for word in word_list: # calculate the tfidf for all the word in the query 
        tf=c[word]/string_len
        idf= math.log(documents_number / documents_with_word[word] ,10) 
        res_vec.append(tf*idf) 
    return res_vec

In [358]:
## Process the query
def search_query(): #the search engine is similar to the previous one, but in this case we use it to filter the list of
                    # documents that contains all the words of the query. We will apply the cosine similarity function to this books
    query= str(input('Insert the query '))
    final_query=cleaner(query)
    word_list=final_query.split()
    tf_idf=find_tfidf(word_list)
    key_list=[]
    for word in word_list:
        key=df_voc[df_voc.word==word].id.iloc[0]
        key_list.append(key)
    try:
        result=new_dic[key_list[0]]
    except:
        print('No results found for the input query')
        return 0,0,0
    prova=[]
    for item in key_list:
        result=result.intersection(new_dic[item])
        print(item)
    return result,key_list,tf_idf  #it returns the set of documents, the words in the query, and the tf_idf vector of the query

The purpose of this function is to allow the user to search for key words that could identify some books. Please avoid to insert only stopwords.

In [366]:
res,key_list,tf_vector=search_query()   # call the function

Insert the query hard love pain life
2
19024
14338
1699


In [268]:
from sklearn.metrics.pairwise import cosine_similarity 

In [372]:
result_list=[] 
for book in res: # for each book in result we create the tfidf_vector. The lenght of the vector is equal to the length 
                # of the tfidf_vector of the query
    vec=[]
    for key in key_list:
        data_frame= pd.DataFrame(inverted_index_tfidf[key])#convert the values of the key in a dataframe 
        data_frame.columns=['tf','doc']
        vec.append(data_frame[data_frame.doc== book].tf.iloc[0]) # search the tfidf value filtering the dataframe 
   
    similarity= cos_similarity(tf_vector,vec) # for each book we apply the cosine similarity
    result_list.append((similarity, book)) # store the result in a list

In [373]:
final_list=heapq.nlargest(4,result_list)  # get the top 4 results 

In [374]:
def find_link(index):  # find the link to the book using its html document 
    ind=str(index)
    if(index%100==0):
        folder=int(ind[0:3])-1
    else:
        folder=ind[0:3]
    HtmlFile = open('books_3/html_books_'+str(folder)+'/article_{}.html'.format(ind), 'r', encoding='utf-8')
    source_code = HtmlFile.read()

    book = soup(source_code, 'html.parser')

    book_link = book.find('link')
    
    return book_link['href']  #return the link

In [375]:
result_dataframe= pd.DataFrame(columns=['Title','Plot','Url','Similarity'])
title_list=[]
plot_list=[]
url_list=[]
sim_list=[]
for similar,index in final_list:  # for each book we print the title, the plot, the url and the cosine similarity
    df = pd.read_csv("books_3_tsv/article_"+str(index)+".tsv", sep='\t', encoding='utf-8' )
    plot_split=re.split(r'\W+', str(df['complete_plot'].iloc[0]))
    title=df['book_title'].iloc[0]
    list_to_remove = ['b','br','span', 'one' , 'id', 'none','[',']']
    clean_more = [w for w in plot_split if w not in list_to_remove] 
    clean_more_2 = [w for w in clean_more if  not (re.findall(re.compile(r'freetext'),w)
                                                   or re.findall(re.compile(r'\d'),w)) ]
    url=find_link(index)
    title_list.append(title)
    plot_list.append(' '.join(clean_more_2))
    url_list.append(url)
    sim_list.append(similar)
result_dataframe['Title'] = title_list
result_dataframe['Plot'] = plot_list
result_dataframe['Url'] = url_list
result_dataframe['Similarity']=sim_list
#print(title,' '.join(clean_more_2),url, similar)
#print('\n\n\n')

In [376]:
result_dataframe

,Title,Plot,Url,Similarity
0,Unbearable Lightness: A Story of Loss and Gain,I didn t decide to become anorexic It snuck u...,https://www.goodreads.com/book/show/9219901-un...,0.977755
1,Most of All You,i A broken woman i Crystal learned long ago t...,https://www.goodreads.com/book/show/32854499-m...,0.972176
2,A Drop in the Ocean,On her birthday Anna Fergusson Boston neurosc...,https://www.goodreads.com/book/show/30129954-a...,0.967638
3,An Unexpected Blessing,i This is an alternate cover edition of ISBN ...,https://www.goodreads.com/book/show/52374459-a...,0.956257
